In [2]:
# All imports

import import_ipynb
import plotly.graph_objects as go
import plotly.express as px
import folium
import pandas as pd

from opencage.geocoder import OpenCageGeocode
from siads_593_milestone1.deepnote.Data_Collection_And_Cleaning import (
    primary_dataset_cleaning, 
    secondary_dataset_cleaning, 
    tertiary_dataset_cleaning,  
    final_merge_df
    )

importing Jupyter notebook from /root/work/siads_593_milestone1/deepnote/Data_Collection_And_Cleaning.ipynb


We want to create a visualization that places a circle for each station on a map of Chicago, and sized according to ridership. Considering we have 23 years of ridership data, we made the size of the circle related to the monthly total station entries for two months, the historical maximum ridership and historical minimum ridership for each station. 

In [3]:
#Functions to assist with mapping

def get_high(df):
    return df.loc[df['MONTHTOTAL'].idxmax()]

def get_low(df):
    return df.loc[df['MONTHTOTAL'].idxmin()]

def normalize_radius(monttotal, max_value=700000, max_radius=250):
    return max(1, int((monttotal / max_value) * max_radius))

In [4]:
def map_visualization(df):

    max_df = df.groupby('STATION_DESCRIPTIVE_NAME').apply(get_high).reset_index(drop=True)
    min_df = df.groupby('STATION_DESCRIPTIVE_NAME').apply(get_low).reset_index(drop=True)

    m = folium.Map(location=[41.8781, -87.6298], zoom_start=12, tiles='CartoDB positron')

    # Add blue circles for max_df
    for idx, row in max_df.iterrows():
        radius = normalize_radius(row['MONTHTOTAL'])
        folium.Circle(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color='red',
            fill=True,
            fill_color='red',
            tooltip=folium.Tooltip(f"Station: {row['STATION_DESCRIPTIVE_NAME']}<br>MONTH_BEGINNING: {row['MONTH_BEGINNING']}<br>MONTHTOTAL: {row['MONTHTOTAL']}")
        ).add_to(m)

    # Add red circles for min_df
    for idx, row in min_df.iterrows():
        radius = normalize_radius(row['MONTHTOTAL'])
        folium.Circle(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color='blue',
            fill=True,
            fill_color='blue',
            tooltip=folium.Tooltip(f"Station: {row['STATION_DESCRIPTIVE_NAME']}<br>MONTH_BEGINNING: {row['MONTH_BEGINNING']}<br>MONTHTOTAL: {row['MONTHTOTAL']}")
        ).add_to(m)
    
    return m

In [5]:
#Visualizing Map
df = final_merge_df
map_visualization(df)

In [6]:
def stacked_bar_vis(df = secondary_dataset_cleaning()):

    df_melted = df.melt(id_vars = 'ZIP', var_name = 'Year', value_name = 'Population')

    # Create the stacked bar chart
    fig = px.bar(df_melted, 
                x = 'Year', 
                y = 'Population', 
                color = 'ZIP', 
                title = 'Population by ZIP Code Over Years',
                labels = {'Population': 'Population', 'Year': 'Year', 'ZIP': 'ZIP Code'},
                hover_data = ['Year', 'ZIP', 'Population'])

    fig.show()

In [7]:
stacked_bar_vis()

In [8]:
def line_vis(df = final_merge_df):
    df['MONTH_BEGINNING'] = pd.to_datetime(df['MONTH_BEGINNING'])

    zip_codes = df['zip_code'].unique()

    fig = go.Figure()

    for zip_code in zip_codes:
        filtered_df = df[df['zip_code'] == zip_code]
        for station in filtered_df['STATION_DESCRIPTIVE_NAME'].unique():
            station_df = filtered_df[filtered_df['STATION_DESCRIPTIVE_NAME'] == station]
            fig.add_trace(go.Scatter(
                x=station_df['MONTH_BEGINNING'], 
                y=station_df['MONTHTOTAL'], 
                mode='lines', 
                name=f"{station} (ZIP: {zip_code})",
                visible=False
            ))

    buttons = []
    for zip_code in zip_codes:
        buttons.append(dict(
            method='update',
            label=str(zip_code),
            args=[{'visible': [str(zip_code) in trace.name for trace in fig.data]},
                {'title': f'Monthly Total Rides for ZIP {zip_code}'}]
        ))

    buttons.append(dict(
        method='update',
        label='All',
        args=[{'visible': [True] * len(fig.data)},
            {'title': 'Monthly Total Rides for All ZIP Codes'}]
    ))

    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
        }],
        title='Monthly Total Rides per Station',
        xaxis_title='Month',
        yaxis_title='Monthly Total Rides',
        hovermode='x unified'
    )

    fig.show()

In [9]:
line_vis()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7862ac62-05ae-412f-9bc1-59d586e87203' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>